In [38]:
# Python实现 Boosting Tree
from collections import defaultdict
import numpy as np

class BoostingTree:
    def __init__(self, error=1e-2):
        self.error = error # 误差值
        self.candidate_splits = [] # 候选切分点
        self.split_index = defaultdict(tuple) # 要多次切分数据集，预先存储，切分后数据点的索引
        self.split_list = [] # 最终各个基本回归树的切分点
        self.cl_list = [] # 切分点左侧区域
        self.cr_list = [] # 切分点右侧区域
        self.N = None # 数组元素个数
        self.n_split = None # 切分点个数
        
    # 切分数组函数
    def split_arr(self, X_data):
        self.N = X_data.shape[0]
        # 候选切分点--前后两个数的中间值
        for i in range(1, self.N):
            self.candidate_splits.append((X_data[i][0] + X_data[i-1][0]) / 2)
        self.n_split = len(self.candidate_splits)
        # 切成两部分
        for split in self.candidate_splits:
            left_index = np.where(X_data[:, 0] <= split)[0] # 返回切分点左侧点的下标
            right_index = np.where(X_data[:, 0] > split)[0]
            self.split_index[split] = (left_index, right_index)
        return
    
    # 计算每个切分点的误差
    def caculate_error(self, split, y_result):
        left, right = self.split_index[split]
        left = y_result[left]
        right = y_result[right]
        
        c1 = np.sum(left) / len(left) # 左均值
        c2 = np.sum(right) / len(right) # 右均值
        y_res_left = left - c1
        y_res_right = right - c2
        res = np.hstack([y_res_left, y_res_right]) # 数据拼接
        res_square = np.apply_along_axis(lambda x:x ** 2, 0, res).sum() # 将res第0维的数值平方后聚合起来
        return res_square, c1, c2
    
    # 获得最佳切分点，并返回对应的残差
    def best_split(self, y_result):
        # 默认第一个为最佳切分点
        best_split = self.candidate_splits[0]
        min_result_square, best_c1, best_c2 = self.caculate_error(best_split, y_result)
        
        for i in range(1, self.n_split):
            res_square, c1, c2 = self.caculate_error(self.candidate_splits[i], y_result)
            if res_square < min_result_square:
                best_split, min_result_square, best_c1, best_c2 = self.candidate_splits[i], res_square, c1, c2
        
        self.split_list.append(best_split)
        self.cl_list.append(best_c1)
        self.cr_list.append(best_c2)
        return
    
    # 基于当前组合树，预测X的输出值
    def predict_x(self, X):
        res = 0
        print("hello")
        for split, c1, c2 in zip(self.split_list, self.cl_list, self.cr_list):
            pre = res
            if X < split:
                res += c1
            else:
                res += c2
            print("split:{}, c1:{}, c2:{}, changing:{}".format(split, c1, c2, res-pre))
        return res
    
    # 每添加一棵回归树，就要更新y，即基于当前组合回归树的预测残差
    def update_y(self, X_data, y_data):
        y_res = []
        for X, y in zip(X_data, y_data):
            y_res.append(y - self.predict_x(X[0])) # 残差
        y_res = np.array(y_res)
        print("updated residual:\n", np.round(y_res, 2)) # 输出每次拟合数据后的残差
        res_square = np.apply_along_axis(lambda x:x ** 2, 0, y_res).sum()
        return y_res, res_square
    
    def fit(self, X_data, y_data):
        # 切分数组
        self.split_arr(X_data)
        y_res = y_data
        epoch = 1
        while True:
            # 获得最佳切分点，并返回对应的残差
            self.best_split(y_res)
            y_res, res_square = self.update_y(X_data, y_data)
            print("第{}轮: 平方损失误差为{}\n".format(epoch, res_square))
            epoch += 1
            if res_square < self.error:
                break
        return

In [39]:
# 使用《统计学习方法》里的例子测试
data = np.array([[1, 5.56], [2, 5.70], [3, 5.91], [4, 6.40], [5, 6.80],
                     [6, 7.05], [7, 8.90], [8, 8.70], [9, 9.00], [10, 9.05]])
X_data = data[:, :-1]
y_data = data[:, -1]
bt = BoostingTree(error=0.18)
bt.fit(X_data, y_data)
print('切分点：', bt.split_list)
print('切分点左区域取值:', np.round(bt.cl_list,2))
print('切分点右区域取值:', np.round(bt.cr_list,2))

hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:6.236666666666667
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:6.236666666666667
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:6.236666666666667
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:6.236666666666667
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:6.236666666666667
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:6.236666666666667
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:8.912500000000001
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:8.912500000000001
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:8.912500000000001
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:8.912500000000001
updated residual:
 [-0.68 -0.54 -0.33  0.16  0.56  0.81 -0.01 -0.21  0.09  0.14]
第1轮: 平方损失误差为1.9300083333333335

hello
s

In [41]:
print("d")
print(bt.predict_x(4))

d
hello
split:6.5, c1:6.236666666666667, c2:8.912500000000001, changing:6.236666666666667
split:3.5, c1:-0.513333333333334, c2:0.219999999999999, changing:0.21999999999999886
split:6.5, c1:0.1466666666666668, c2:-0.2200000000000002, changing:0.1466666666666665
split:4.5, c1:-0.16083333333333316, c2:0.1072222222222227, changing:-0.1608333333333336
split:6.5, c1:0.0714814814814817, c2:-0.10722222222222255, changing:0.0714814814814817
split:2.5, c1:-0.1506481481481483, c2:0.0376620370370373, changing:0.03766203703703752
6.551643518518518
